In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/EnKF/")

"/media/mat/HDD/EnKF/Project.toml"

In [2]:
using Revise
using EnKF
using Distributions
using LinearAlgebra

In [25]:
@time A = MvNormal(ones(5), I)
@time out = initialize(10, A)

  0.000004 seconds (6 allocations: 336 bytes)
  0.000007 seconds (29 allocations: 3.031 KiB)


EnsembleState{10,5,Array{Float64,1}}(Array{Float64,1}[[0.236911, -0.0859332, 2.76449, 1.58524, 1.82615], [0.849639, 1.12062, 1.09585, 0.797986, 2.39562], [-0.416957, 1.23377, 2.06878, 0.330737, 1.95477], [1.91354, 2.28568, 1.98725, -0.51684, 2.29306], [-0.454927, 1.55253, 2.74027, 0.379721, -0.974558], [-0.758032, 2.78702, 1.99991, -0.371311, -0.210057], [-0.465391, -0.0243325, 3.77458, -0.398304, 2.94738], [0.0622524, 0.360558, -0.302994, 0.621546, 1.72012], [1.05702, 1.07318, 1.66801, 1.41134, -0.294342], [0.826141, 0.736847, -1.22933, 1.15078, 0.400031]])

In [26]:
deviation(out)

EnsembleState{10,5,Array{Float64,1}}(Array{Float64,1}[[-0.0481084, -1.18993, 1.10781, 1.08615, 0.620333], [0.564619, 0.0166256, -0.560828, 0.298896, 1.1898], [-0.701976, 0.12978, 0.412094, -0.168353, 0.748951], [1.62852, 1.18169, 0.330571, -1.01593, 1.08724], [-0.739946, 0.448535, 1.08359, -0.119368, -2.18038], [-1.04305, 1.68302, 0.343229, -0.870401, -1.41587], [-0.750411, -1.12833, 2.11789, -0.897394, 1.74156], [-0.222767, -0.743437, -1.95968, 0.122456, 0.514301], [0.772001, -0.0308115, 0.0113266, 0.912254, -1.50016], [0.541121, -0.367147, -2.88601, 0.651694, -0.805786]])

In [27]:
mean(out)

5-element Array{Float64,1}:
 -4.4408920985006264e-17
  9.992007221626409e-17 
  2.220446049250313e-16 
  4.4408920985006264e-17
  0.0                   

In [8]:
Add = AdditiveInflation(5)

AdditiveInflation{5}(IsoNormal(
dim: 5
μ: [0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)
)

In [13]:
out = EnsembleState(10,5)

EnsembleState{10,5,Array{Float64,1}}(Array{Float64,1}[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]])

In [14]:
@time Add(out)

  0.000013 seconds (34 allocations: 1.875 KiB)


EnsembleState{10,5,Array{Float64,1}}(Array{Float64,1}[[-1.62329, 1.80771, -0.435176, 0.240917, -1.52402], [0.869775, -0.515919, 0.111825, 0.0606616, 1.46286], [1.72131, 0.326619, -0.164714, -1.2315, -1.44447], [2.08271, -0.83294, 0.507098, 0.339488, -2.08162], [-0.647263, 0.536572, -2.19642, 0.701106, -0.0583067], [-0.10702, 0.541239, 0.0840684, -1.3913, -0.682701], [-0.432507, 1.11047, -0.37265, -0.371761, -0.902206], [0.0897896, 0.57983, 0.423491, -0.579849, -0.507468], [-0.174747, 2.16469, -0.320401, 0.590347, -0.720571], [0.823658, 1.0179, 0.417184, 1.37398, -0.165798]])

In [15]:
Mul = MultiplicativeInflation(5)

MultiplicativeInflation{5}(1.0)

In [16]:
out = EnsembleState(10,5)

EnsembleState{10,5,Array{Float64,1}}(Array{Float64,1}[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]])

In [17]:
Mul(out)

EnsembleState{10,5,Array{Float64,1}}(Array{Float64,1}[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]])

In [ ]:
out = EnsembleState(10,5)

In [20]:
MulAdd = MultiAdditiveInflation(5)

MultiAdditiveInflation{5}(1.0, IsoNormal(
dim: 5
μ: [0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)
)

In [21]:
MulAdd.β = 1.05

1.05

In [22]:
MulAdd(out)

EnsembleState{10,5,Array{Float64,1}}(Array{Float64,1}[[-0.493131, -1.38874, 0.231262, -0.4716, 0.0677534], [1.47857, -0.0531732, 2.64909, -0.268656, -1.24905], [1.34614, -2.35271, -0.552836, -0.317407, 1.28813], [0.31759, -0.60549, 0.29785, 0.52047, -0.257745], [-0.0521986, -2.14691, -1.40009, -0.495878, -1.1197], [0.0394378, 1.97797, -0.909857, 1.1848, -0.292952], [-0.918117, 1.63462, -1.45435, -0.141762, -0.056872], [0.583282, -2.1044, 1.32379, 0.700178, -2.25503], [0.0985249, -1.38451, -0.770304, -0.255218, -1.26062], [-1.24293, -0.263326, -0.288099, 2.33894, -0.671788]])